In [181]:
import tensorflow as tf
import numpy as np
import albumentations as A
import cv2
from functools import partial

In [ ]:
# hyperparameters 
NUM_CLASSES = 11
STRIDES = np.array([8,16,32])
ANCHORS =(1.25,1.625, 2.0,3.75, 4.125,2.875, 1.875,3.8125, 3.875,2.8125, 3.6875,7.4375, 3.625,2.8125, 4.875,6.1875, 11.65625,10.1875)
ANCHORS = np.array(ANCHORS).reshape(3,3,2) # 3개의 스케일
ANCHORS[0]/

In [5]:
TRAIN_TFRECORD_PATH = "./EDA/train_dataset.tfrecord"
BATCH_SIZE = 4
IMAGE_SIZE = 416
scale = 1.1

ANCHORS = [
    [[10, 13], [16, 30], [33, 23]],
    [[30, 61], [62, 45], [59,119]],
    [[116, 90], [156, 198], [373, 326]]
]

NUM_CLASSES = 3
TRAIN_INPUT_SIZE = 416
STRIDES = tf.constant([8, 16, 32],dtype=tf.float32)
TRAIN_OUTPUT_SIZE = TRAIN_INPUT_SIZE //STRIDES
ANCHOR_PER_SCALE = 3
MAX_BBOX_PER_SCALE = 100
IOU_ANCHOR_THRESH = 0.3
OUTPUT_LEVELS = STRIDES.shape[0]

# IoU Loss

In [3]:
# testing (should be better than giou)
def bbox_ciou(boxes1, boxes2):
    boxes1_coor = tf.concat([boxes1[..., :2] - boxes1[..., 2:] * 0.5,
                        boxes1[..., :2] + boxes1[..., 2:] * 0.5], axis=-1)
    boxes2_coor = tf.concat([boxes2[..., :2] - boxes2[..., 2:] * 0.5,
                        boxes2[..., :2] + boxes2[..., 2:] * 0.5], axis=-1)

    left = tf.maximum(boxes1_coor[..., 0], boxes2_coor[..., 0])
    up = tf.maximum(boxes1_coor[..., 1], boxes2_coor[..., 1])
    right = tf.maximum(boxes1_coor[..., 2], boxes2_coor[..., 2])
    down = tf.maximum(boxes1_coor[..., 3], boxes2_coor[..., 3])

    c = (right - left) * (right - left) + (up - down) * (up - down)
    iou = bbox_iou(boxes1, boxes2)

    u = (boxes1[..., 0] - boxes2[..., 0]) * (boxes1[..., 0] - boxes2[..., 0]) + (boxes1[..., 1] - boxes2[..., 1]) * (boxes1[..., 1] - boxes2[..., 1])
    d = u / c

    ar_gt = boxes2[..., 2] / boxes2[..., 3]
    ar_pred = boxes1[..., 2] / boxes1[..., 3]

    ar_loss = 4 / (np.pi * np.pi) * (tf.atan(ar_gt) - tf.atan(ar_pred)) * (tf.atan(ar_gt) - tf.atan(ar_pred))
    alpha = ar_loss / (1 - iou + ar_loss + 0.000001)
    ciou_term = d + alpha * ar_loss

    return iou - ciou_term

In [219]:
def bbox_iou(boxes1, boxes2):
    """
    Anchor box 특성 상 중심점이 정해져있기 때문에 w, h 만을 가지고 iou를 계산
    """
    
    boxes1_area = boxes1[..., 2] * boxes1[..., 3]
    boxes2_area = boxes2[..., 2] * boxes2[..., 3]

    boxes1 = tf.concat([boxes1[..., :2] - boxes1[..., 2:] * 0.5,
                        boxes1[..., :2] + boxes1[..., 2:] * 0.5], axis=-1)
    boxes2 = tf.concat([boxes2[..., :2] - boxes2[..., 2:] * 0.5,
                        boxes2[..., :2] + boxes2[..., 2:] * 0.5], axis=-1)

    left_up = tf.maximum(boxes1[..., :2], boxes2[..., :2])
    right_down = tf.minimum(boxes1[..., 2:], boxes2[..., 2:])

    inter_section = tf.maximum(right_down - left_up, 0.0)
    inter_area = inter_section[..., 0] * inter_section[..., 1]
    union_area = boxes1_area + boxes2_area - inter_area

    return 1.0 * inter_area / union_area

# Anchor + BBox

In [228]:
TRAIN_TFRECORD_PATH = "./EDA/train_dataset.tfrecord"
BATCH_SIZE = 4
IMAGE_SIZE = 416
scale = 1.1

YOLO_ANCHORS = tf.constant([
    [[10, 13], [16, 30], [33, 23]],
    [[30, 61], [62, 45], [59,119]],
    [[116, 90], [156, 198], [373, 326]]
], dtype=tf.int32)

NUM_CLASSES = 3

TRAIN_INPUT_SIZE = 416
STRIDES = tf.constant([8, 16, 32],dtype=tf.int32)
TRAIN_OUTPUT_SIZE = TRAIN_INPUT_SIZE //STRIDES
ANCHOR_PER_SCALE = 3
MAX_BBOX_PER_SCALE = 100
IOU_ANCHOR_THRESH = 0.3
OUTPUT_LEVELS = STRIDES.shape[0]

In [229]:
def iou_height_width(boxes1, boxes2):
    boxes1_area = boxes1[..., 2] * boxes1[..., 3]
    boxes2_area = boxes2[..., 2] * boxes2[..., 3]

    boxes1 = np.concatenate([boxes1[..., :2] - boxes1[..., 2:] * 0.5,
                        boxes1[..., :2] + boxes1[..., 2:] * 0.5], axis=-1)
    boxes2 = np.concatenate([boxes2[..., :2] - boxes2[..., 2:] * 0.5,
                        boxes2[..., :2] + boxes2[..., 2:] * 0.5], axis=-1)

    left_up = np.maximum(boxes1[..., :2], boxes2[..., :2])
    right_down = np.minimum(boxes1[..., 2:], boxes2[..., 2:])

    inter_section = np.maximum(right_down - left_up, 0.0)
    inter_area = inter_section[..., 0] * inter_section[..., 1]
    union_area = boxes1_area + boxes2_area - inter_area

    return 1.0 * inter_area / union_area

In [235]:
boxes = np.array([ [101,212,33,43],[142,21,31,61],[311,362,32,41] ])
classes = np.array([0,1,2])

preprocess_true_boxes(boxes, classes)

0
<class 'int'>


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [234]:
def preprocess_true_boxes(bboxes, classes):
    output_levels = OUTPUT_LEVELS
    train_output_sizes = TRAIN_OUTPUT_SIZE
    max_bbox_per_scale = MAX_BBOX_PER_SCALE
    num_classes = NUM_CLASSES
    anchor_per_scale = ANCHOR_PER_SCALE
    anchors = YOLO_ANCHORS
    iou_anchor_threshold = IOU_ANCHOR_THRESH
    strides = STRIDES
    
    gt_data = np.concatenate( [bboxes, classes[:,np.newaxis]], axis=1)
    label = [
        np.zeros([52, 52, anchor_per_scale, 5 + num_classes]),
        np.zeros([26, 26, anchor_per_scale, 5 + num_classes]),
        np.zeros([13, 13, anchor_per_scale, 5 + num_classes])
    ]

    bboxes_xywh = [
        np.zeros((max_bbox_per_scale, 4)),
        np.zeros((max_bbox_per_scale, 4)),
        np.zeros((max_bbox_per_scale, 4))
    ]
    
    bbox_count = np.zeros((OUTPUT_LEVELS,))

    
    for bbox in gt_data:
        bbox_coor = bbox[:4]
        bbox_class_ind =bbox[4]

        onehot = np.zeros(num_classes, dtype=np.float32)
        onehot[bbox_class_ind] = 1.0
        uniform_distribution = np.full(num_classes, 1.0 / num_classes)
        deta = 0.01
        smooth_onehot = onehot * (1 - deta) + deta * uniform_distribution

        bbox_xywh = np.concatenate([(bbox_coor[2:] + bbox_coor[:2]) * 0.5, bbox_coor[2:] - bbox_coor[:2]], axis=-1)
        bbox_xywh_scaled = 1.0 * bbox_xywh[np.newaxis, :] / strides[:, np.newaxis]

        iou = []
        exist_positive = False
        for i in range(output_levels):#range(3):
            anchors_xywh = np.zeros((anchor_per_scale, 4))
            anchors_xywh[:, 0:2] = np.floor(bbox_xywh_scaled[i, 0:2]).astype(np.float32) + 0.5
            anchors_xywh[:, 2:4] = anchors[i]

            iou_scale = iou_height_width(bbox_xywh_scaled[i][np.newaxis, :], anchors_xywh)
            iou.append(iou_scale)
            iou_mask = iou_scale > iou_anchor_threshold

            if np.any(iou_mask):
                xind, yind = np.floor(bbox_xywh_scaled[i, 0:2]).astype(np.float32)

                label[i][yind, xind, iou_mask, :] = 0
                label[i][yind, xind, iou_mask, 0:4] = bbox_xywh
                label[i][yind, xind, iou_mask, 4:5] = 1.0
                label[i][yind, xind, iou_mask, 5:] = smooth_onehot

                bbox_ind = int(bbox_count[i] % max_bbox_per_scale)
                bboxes_xywh[i][bbox_ind, :4] = bbox_xywh
                bbox_count[i] += 1
                exist_positive = True

        if not exist_positive:
            best_anchor_ind = np.argmax(np.array(iou).reshape(-1), axis=-1)
            best_detect = int(best_anchor_ind / anchor_per_scale)
            print(best_detect)
            print(type(best_detect))
            best_anchor = int(best_anchor_ind % anchor_per_scale)
            xind, yind = np.floor(bbox_xywh_scaled[best_detect, 0:2]).astype(np.float32)

            label[best_detect][yind, xind, best_anchor, 0:4] = bbox_xywh
            label[best_detect][yind, xind, best_anchor, 4:5] = 1.0
            label[best_detect][yind, xind, best_anchor, 5:] = smooth_onehot

            bbox_ind = int(bbox_count[best_detect] % max_bbox_per_scale)
            bboxes_xywh[best_detect][bbox_ind, :4] = bbox_xywh
            bbox_count[best_detect] += 1

    label_sbbox, label_mbbox, label_lbbox = label
#     sbboxes, mbboxes, lbboxes = bboxes_xywh
#     return label_sbbox, label_mbbox, label_lbbox, sbboxes, mbboxes, lbboxes
    return label_sbbox, label_mbbox, label_lbbox

In [227]:
def _process_target(train):
    def process_target(gt_boxes, classes):
        label_sbbox, label_mbbox, label_lbbox = tf.numpy_function(func=preprocess_true_boxes, inp=[gt_boxes, classes],Tout=[tf.float32, tf.float32, tf.float32])
        return label_sbbox, label_mbbox, label_lbbox
    return process_target

# Augmentation

In [159]:
train_transforms = A.Compose(
    [
        A.LongestMaxSize(max_size=int(IMAGE_SIZE * scale)),
        A.PadIfNeeded(
            min_height=int(IMAGE_SIZE * scale),
            min_width=int(IMAGE_SIZE * scale),
            border_mode=cv2.BORDER_CONSTANT,
        ),
        A.RandomCrop(width=IMAGE_SIZE, height=IMAGE_SIZE),
        A.ColorJitter(brightness=0.6, contrast=0.6, saturation=0.6, hue=0.6, p=0.4),
        A.OneOf(
            [
                A.ShiftScaleRotate(
                    rotate_limit=20, p=0.5, border_mode=cv2.BORDER_CONSTANT
                ),
                A.IAAAffine(shear=15, p=0.5, mode="constant"),
            ],
            p=1.0,
        ),
        A.HorizontalFlip(p=0.5),
        A.Blur(p=0.1),
        A.CLAHE(p=0.1),
        A.Posterize(p=0.1),
        A.ToGray(p=0.1),
        A.ChannelShuffle(p=0.05),
    ],
    bbox_params=A.BboxParams(format="pascal_voc", min_visibility=0.4),
)

test_transforms = A.Compose(
    [
        A.LongestMaxSize(max_size=IMAGE_SIZE),
        A.PadIfNeeded(
            min_height=IMAGE_SIZE, min_width=IMAGE_SIZE, border_mode=cv2.BORDER_CONSTANT
        ),
    ],
    bbox_params=A.BboxParams(format="pascal_voc", min_visibility=0.4),
)

In [160]:
def aug_fn(image, bbox, train):
    data = {"image":image,"bboxes":bbox}
    if train:
        aug_data = train_transforms(**data)
    else:
        aug_data = test_transforms(**data)
    aug_img = aug_data["image"]
    aug_bbox = aug_data["bboxes"]
    aug_img = tf.cast(aug_img/255.0, tf.float32)
    aug_bbox = tf.cast(aug_bbox, tf.float32)
    return aug_img, aug_bbox

In [161]:
def _process_aug(train):
    def process_aug(image, bbox):
        aug_img, aug_bbox = tf.numpy_function(func=aug_fn, inp=[image, bbox, train],Tout=[tf.float32, tf.float32])
        return aug_img, aug_bbox
    return process_aug

# Load TFRecord

In [173]:
def _parse_tfrecord(train):

    def parse_tfrecord(tfrecord):
        features = {
            'filename': tf.io.FixedLenFeature([], tf.string),
            'height': tf.io.FixedLenFeature([], tf.int64),
            'width': tf.io.FixedLenFeature([], tf.int64),
            'classes': tf.io.VarLenFeature(tf.int64),
#             'objectness': tf.io.VarLenFeature(tf.int64),
            'target_id': tf.io.VarLenFeature(tf.int64),
            'x_min': tf.io.VarLenFeature(tf.float32),
            'y_min': tf.io.VarLenFeature(tf.float32),
            'x_max': tf.io.VarLenFeature(tf.float32),
            'y_max': tf.io.VarLenFeature(tf.float32),
            'image_raw': tf.io.FixedLenFeature([], tf.string),
        }

        parsed_example = tf.io.parse_single_example(tfrecord, features)
        img = tf.image.decode_jpeg(parsed_example['image_raw'], channels=3)
        
        img = tf.cast(img, tf.float32)
        width = tf.cast(parsed_example['width'], tf.float32)
        height = tf.cast(parsed_example['height'], tf.float32)
        
        labels = tf.sparse.to_dense(parsed_example['classes'])
        labels = tf.cast(labels, tf.float32)
#         objectness = tf.sparse.to_dense(parsed_example['objectness'])
#         objectness = tf.cast(objectness, tf.float32)
        target_id = tf.sparse.to_dense(parsed_example['target_id'])
        target_id = tf.cast(target_id, tf.float32)
        
        bboxes = tf.stack(
            [
                tf.sparse.to_dense(parsed_example['x_min']),
                tf.sparse.to_dense(parsed_example['y_min']),
                tf.sparse.to_dense(parsed_example['x_max']),
                tf.sparse.to_dense(parsed_example['y_max']),
            ],axis=1)
        
        img, bboxes = _process_aug(train)(img, bboxes)
          
        label_sbbox, label_mbbox, label_lbbox = _process_target(train)(bboxes, labels)
        
        img.set_shape( (416,416,3) )
#         label_sbbox.set_shape ((416,416,3))
#         label_mbbox.set_shape ((416,416,3))
#         label_lbbox.set_shape ((416,416,3))
        return img,  label_sbbox, label_mbbox, label_lbbox

    return parse_tfrecord

In [170]:
def load_tfrecord_dataset(tfrecord_name, train=True, buffer_size=1024):
    raw_dataset = tf.data.TFRecordDataset(tfrecord_name)
    raw_dataset = raw_dataset.cache() # 처음 불러오게 되면 캐시메모리로 저장
    
    if train:
        raw_dataset = raw_dataset.repeat() # 이 데이터를 몇번 iteration할건지 지정, 지정하지 않으면 -1로 되며 계속 반복
        raw_dataset = raw_dataset.shuffle(buffer_size=buffer_size) # 데이터 셔플
        
    # 데이터 읽기 시작
    raw_dataset = raw_dataset.map(_parse_tfrecord(train), num_parallel_calls=tf.data.experimental.AUTOTUNE)  

    raw_dataset = raw_dataset.batch(BATCH_SIZE, drop_remainder=True)
    raw_dataset = raw_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return raw_dataset

# 호출

In [171]:
train_dataset = load_tfrecord_dataset(
    tfrecord_name=TRAIN_TFRECORD_PATH,
    train=True
)

In [180]:
for i in train_dataset.take(1):
    break

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_4_device_/job:localhost/replica:0/task:0/device:CPU:0}} Feature: filename (data type: string) is required but could not be found.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]] [Op:IteratorGetNext]

In [48]:
def load_dataset(boxes, train=True, buffer_size=1024):
    if train:
        dataset = load_tfrecord_dataset(
            tfrecord_name=TRAIN_TFRECORD_PATH,
            train=train,
            boxes=boxes,
            buffer_size=buffer_size)
    else:
        dataset = load_tfrecord_dataset(
            tfrecord_name=VALID_TFRECORD_PATH,
            train=train,
            boxes=boxes,
            buffer_size=buffer_size)
    return dataset